PRE-PROCESSING

**“DSC{20-dsc-0324-bcn} CLUSTERING DS MARKET TFM -{Anchaoui/Viana/Reguillón/Araiza}”**

# Task 1 - PREPROCESSING(YEAR/MONTH - Groupby)

> El DS resultante de este preprocessing se ha utilizado como origen de datos del BI y del cálculo de los clústeres por ítems!


# Librerias

In [ ]:
import sys
ENV_COLAB = 'google.colab' in sys.modules
if ENV_COLAB:
      !pip install catboost
      !pip install ipywidgets
      !pip install ydata_profiling
      !jupyter nbextension enable --py widgetsnbextension
      print('Environment: Google Colab')

Enabling notebook extension jupyter-js-widgets/extension...
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json
Paths used for configuration of notebook: 
    	
      - Validating: OK
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json
Environment: Google Colab


In [ ]:
import sys
import time
import warnings

# imports best practice pandas
import os
import numpy as np
import pandas as pd

# imports best practice sklearn
import sklearn
from sklearn import set_config

from sklearn.tree import DecisionTreeClassifier

# transformers
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder

# pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# model selection e hiperparameters optimization
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, RepeatedKFold, KFold

from sklearn.metrics import accuracy_score

#Plot and profiling
import matplotlib.pyplot as plt
import seaborn as sns
from ydata_profiling import ProfileReport
from catboost import CatBoostClassifier, Pool
from ydata_profiling import ProfileReport

pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows', None)

import warnings
warnings.filterwarnings("ignore")

# Importar train data

In [ ]:
ENV_COLAB = 'google.colab' in sys.modules
if ENV_COLAB:
  from google.colab import drive
  drive.mount('/content/drive')
else:
  print('Environment: Local')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
ENV_COLAB = 'google.colab' in sys.modules
path='/content/drive/MyDrive/TFM/CSV'
if ENV_COLAB:
  df_p = pd.read_csv(path+'/item_prices.csv')
  df_s=pd.read_csv(path+'/item_sales.csv')
  df_c=pd.read_csv(path+'/daily_calendar_with_events.csv')
else:
  df_p = pd.read_csv(r'CSV/item_prices.csv')
  df_s=pd.read_csv(r'CSV/item_sales.csv')
  df_c=pd.read_csv(r'CSV/daily_calendar_with_events.csv')

# EDA: Analisis Exploratorio

#  EDA/Preprocessing: Tabla de Precios

In [ ]:
# Hacemos un Profiling para la tabla de precios

profile_price = ProfileReport(df_p, title='Profiling Report - Price', minimal=True)
profile_price.to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
df_p2=df_p.copy()

In [ ]:
df_p2[df_p2['yearweek'].isnull()].head(3)

,item,category,store_code,yearweek,sell_price
149,ACCESORIES_1_001,ACCESORIES,NYC_1,NaN,11.1454
150,ACCESORIES_1_001,ACCESORIES,NYC_1,NaN,11.1454
151,ACCESORIES_1_001,ACCESORIES,NYC_1,NaN,11.1454


In [ ]:
#Hay mas de una semana seguida con valores nulos, con lo cual hay que buscar una manera de imputarlos de manera consecutiva
df_p2.iloc[147:158]

,item,category,store_code,yearweek,sell_price
147,ACCESORIES_1_001,ACCESORIES,NYC_1,201616.0,11.1454
148,ACCESORIES_1_001,ACCESORIES,NYC_1,201617.0,11.1454
149,ACCESORIES_1_001,ACCESORIES,NYC_1,NaN,11.1454
150,ACCESORIES_1_001,ACCESORIES,NYC_1,NaN,11.1454
151,ACCESORIES_1_001,ACCESORIES,NYC_1,NaN,11.1454
152,ACCESORIES_1_001,ACCESORIES,NYC_1,NaN,11.1454
153,ACCESORIES_1_001,ACCESORIES,NYC_1,NaN,11.1454
154,ACCESORIES_1_001,ACCESORIES,NYC_1,NaN,11.1454
155,ACCESORIES_1_001,ACCESORIES,NYC_1,NaN,11.1454
156,ACCESORIES_1_001,ACCESORIES,NYC_1,NaN,11.1454


In [ ]:
last_valid_value = None
for i in df_p2.index:
  if pd.isnull(df_p2.loc[i, 'yearweek']):
    if last_valid_value:
      df_p2.loc[i, 'yearweek'] = last_valid_value + 1
      last_valid_value += 1 # incremento para imputar los nulos consecutivos y sumar 1 semana al valor anterior
    else:
      # Como el primer registro de la tabla no es nulo, no hace falta hacer ninguna acción.
      pass
  else:
    last_valid_value = df_p2.loc[i, 'yearweek']

In [ ]:
#limpiamos los 0 de la columna de Y/W
df_p2['yearweek'] = df_p2['yearweek'].apply(lambda x: str(x).replace('.0', '') if pd.notnull(x) else x).astype(int)

In [ ]:
# Convertir 'yearweek' en objetos datetime, asumiendo que cada semana comienza el lunes
df_p2['date'] = pd.to_datetime(df_p2['yearweek'].astype(str) + '1', format='%Y%W%w') - pd.Timedelta(days=2) # # Restar 2 días para obtener el sábado anterior, con el analisi de las ventas hemos detectado que las semanas de Yearweek empiezan el sabado

# Extraer mes y año
df_p2['month'] = df_p2['date'].dt.strftime('%m')
df_p2['year'] = df_p2['date'].dt.strftime('%Y')

# Crear columna 'yearmonth'
df_p2['yearmonth'] = df_p2['year'] + df_p2['month']
df_p2.drop(columns=['date','year','month'], inplace=True)

In [ ]:
df_p2 = df_p2.drop_duplicates(keep='first')

In [ ]:
df_p2.isnull().sum()

,0
item,0
category,0
store_code,0
yearweek,0
sell_price,0
yearmonth,0


In [ ]:
df_p2['id']=df_p2['yearmonth'].astype(str) +'_'+df_p2['item'].astype(str) +'_'+ df_p2['store_code'].astype(str) #creamos una pk para unir con precios y calendario
df_p2 = df_p2.sort_values('yearweek')

In [ ]:
df_p2.duplicated().sum()

0

## Group By Precios-- YEAR/MONTH

In [ ]:
df_p2.head(2).T

,1792652,4858854
item,SUPERMARKET_2_037,SUPERMARKET_3_697
category,SUPERMARKET,SUPERMARKET
store_code,NYC_3,BOS_3
yearweek,201105,201105
sell_price,8.208,4.296
yearmonth,201101,201101
id,201101_SUPERMARKET_2_037_NYC_3,201101_SUPERMARKET_3_697_BOS_3


In [ ]:
 gdf_p2 = df_p2.groupby(['yearmonth', 'id']).agg(
     average_price = ('sell_price', np.mean)
 ).reset_index()
 gdf_p2.head(10)



,yearmonth,id,average_price
0,201101,201101_ACCESORIES_1_004_BOS_1,5.7722
1,201101,201101_ACCESORIES_1_004_BOS_2,5.7722
2,201101,201101_ACCESORIES_1_004_NYC_2,5.7722
3,201101,201101_ACCESORIES_1_004_NYC_4,5.7722
4,201101,201101_ACCESORIES_1_004_PHI_1,5.7722
5,201101,201101_ACCESORIES_1_004_PHI_3,5.7722
6,201101,201101_ACCESORIES_1_008_BOS_1,0.6118
7,201101,201101_ACCESORIES_1_008_BOS_2,0.6118
8,201101,201101_ACCESORIES_1_008_BOS_3,0.6384
9,201101,201101_ACCESORIES_1_008_NYC_1,0.6118


In [ ]:
gdf_p2.head(3).T

,0,1,2
yearmonth,201101,201101,201101
id,201101_ACCESORIES_1_004_BOS_1,201101_ACCESORIES_1_004_BOS_2,201101_ACCESORIES_1_004_NYC_2
average_price,5.7722,5.7722,5.7722


In [ ]:
gdf_p2.isnull().sum()

,0
yearmonth,0
id,0
average_price,0


#  EDA/Preprocessing CALENDAR

In [ ]:
#Hacemos un Profiling para la tabla de precios
profile_Calendar = ProfileReport(df_c, title='profile_Calendar', minimal=True)
profile_Calendar.to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
profile_Calendar.to_file("profile_Calendar")

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
df_c.head(10)

,date,weekday,weekday_int,d,event
0,2011-01-29,Saturday,1,d_1,NaN
1,2011-01-30,Sunday,2,d_2,NaN
2,2011-01-31,Monday,3,d_3,NaN
3,2011-02-01,Tuesday,4,d_4,NaN
4,2011-02-02,Wednesday,5,d_5,NaN
5,2011-02-03,Thursday,6,d_6,NaN
6,2011-02-04,Friday,7,d_7,NaN
7,2011-02-05,Saturday,1,d_8,NaN
8,2011-02-06,Sunday,2,d_9,SuperBowl
9,2011-02-07,Monday,3,d_10,NaN


In [ ]:
# borramos la columna EVENTOS 0 relevnate
df_c.drop(columns=['event'], inplace=True)

In [ ]:
# creamos copia para el caledario
df_c1 = df_c.copy()

In [ ]:
# Convertir 'yearweek' en objetos datetime, asumiendo que cada semana comienza el domingo

df_c1['date'] = pd.to_datetime(df_c1['date'])

# Extraer month y year
df_c1['month'] = df_c1['date'].dt.strftime('%m')
df_c1['year'] = df_c1['date'].dt.strftime('%Y')

# Crear columna 'yearmonth'
df_c1['yearmonth'] = df_c1['year'] + df_c1['month']

# Pasar 'yearweek' a datetime
df_c1['date_yw'] = pd.to_datetime(df_c1['date'])

# Calcular el sábado anterior
df_c1['date_yw'] = df_c1['date_yw'] + pd.offsets.Week(weekday=1)

# Extraer año y número de semana (con el sábado como inicio de la semana)
df_c1['yearweek'] = df_c1['date_yw'].dt.strftime('%Y%W').astype(int)


df_c1.drop(columns=['year','month'], inplace=True)

In [ ]:
print(df_c1.head(5))
print(df_c1.tail(5))

        date    weekday  weekday_int    d yearmonth    date_yw  yearweek
0 2011-01-29   Saturday            1  d_1    201101 2011-02-01    201105
1 2011-01-30     Sunday            2  d_2    201101 2011-02-01    201105
2 2011-01-31     Monday            3  d_3    201101 2011-02-01    201105
3 2011-02-01    Tuesday            4  d_4    201102 2011-02-08    201106
4 2011-02-02  Wednesday            5  d_5    201102 2011-02-08    201106
           date    weekday  weekday_int       d yearmonth    date_yw  yearweek
1908 2016-04-20  Wednesday            5  d_1909    201604 2016-04-26    201617
1909 2016-04-21   Thursday            6  d_1910    201604 2016-04-26    201617
1910 2016-04-22     Friday            7  d_1911    201604 2016-04-26    201617
1911 2016-04-23   Saturday            1  d_1912    201604 2016-04-26    201617
1912 2016-04-24     Sunday            2  d_1913    201604 2016-04-26    201617


In [ ]:
df_c1.isnull().sum()

,0
date,0
weekday,0
weekday_int,0
d,0
yearmonth,0
date_yw,0
yearweek,0


#  EDA/Preprocessing SALES

In [ ]:
df_s1 = df_s.copy()

In [ ]:
# hacemos una lista que solo incluya las columnas que empiezan con "d_" y las transponemos para hacer el reshape del dataframe para traer las columnas "d" y su valor

cols = [col for col in df_s1.columns if col.startswith('d_')] # creamos la lista

df_s1 = df_s1.melt(id_vars=[col for col in df_s1.columns if col not in cols],
                   value_vars=cols,
                   var_name='d',
                   value_name='quantity')

In [ ]:
#Hacemos un Profiling para la tabla de sales
profile_sales = ProfileReport(df_s1, title='profile_sales', minimal=True)
profile_sales.to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Buffered data was truncated after reaching the output size limit.

In [ ]:
profile_sales.to_file('profile_sales')

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
# En el profile report vemos que, al hacer el "unpivot" de las columnas de "d_", tenemos registros con venta =0. Quitamos los registros con venta=0
df_s1 = df_s1[df_s1['quantity']!=0]

In [ ]:
df_s1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18550276 entries, 7 to 58327368
Data columns (total 9 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   id          object
 1   item        object
 2   category    object
 3   department  object
 4   store       object
 5   store_code  object
 6   region      object
 7   d           object
 8   quantity    int64 
dtypes: int64(1), object(8)
memory usage: 1.4+ GB


# JOINS

# Join Sales + Calendar

In [ ]:
# hacemos merge para las ventas y el calendario
df_s_C = df_s1.merge(df_c1[['d', 'date_yw','yearweek','yearmonth']], left_on='d', right_on='d', how='left')

In [ ]:
df_s_C.head(3).T

,0,1,2
id,ACCESORIES_1_008_NYC_1,ACCESORIES_1_009_NYC_1,ACCESORIES_1_015_NYC_1
item,ACCESORIES_1_008,ACCESORIES_1_009,ACCESORIES_1_015
category,ACCESORIES,ACCESORIES,ACCESORIES
department,ACCESORIES_1,ACCESORIES_1,ACCESORIES_1
store,Greenwich_Village,Greenwich_Village,Greenwich_Village
store_code,NYC_1,NYC_1,NYC_1
region,New York,New York,New York
d,d_1,d_1,d_1
quantity,12,2,4
date_yw,2011-02-01 00:00:00,2011-02-01 00:00:00,2011-02-01 00:00:00


In [ ]:
df_s_C.rename(columns={'id': 'id_s', 'date_yw': 'date'}, inplace=True)

In [ ]:
#Creamos una pk para unir con precios
df_s_C['id']= df_s_C['yearmonth'].astype(str) + '_' + df_s_C['id_s'].astype(str)


In [ ]:
df_s_C.head(3).T

,0,1,2
id_s,ACCESORIES_1_008_NYC_1,ACCESORIES_1_009_NYC_1,ACCESORIES_1_015_NYC_1
item,ACCESORIES_1_008,ACCESORIES_1_009,ACCESORIES_1_015
category,ACCESORIES,ACCESORIES,ACCESORIES
department,ACCESORIES_1,ACCESORIES_1,ACCESORIES_1
store,Greenwich_Village,Greenwich_Village,Greenwich_Village
store_code,NYC_1,NYC_1,NYC_1
region,New York,New York,New York
d,d_1,d_1,d_1
quantity,12,2,4
date,2011-02-01 00:00:00,2011-02-01 00:00:00,2011-02-01 00:00:00


# Group By Sales+Calendar

In [ ]:
gdf_s_C = df_s_C.groupby(['yearmonth', 'id','id_s','region','store_code','store','department','category','item']).agg(
     Monthly_qty= ('quantity', np.sum)
 ).reset_index()



In [ ]:
gdf_s_C.head(3).T

,0,1,2
yearmonth,201101,201101,201101
id,201101_ACCESORIES_1_004_BOS_1,201101_ACCESORIES_1_004_BOS_2,201101_ACCESORIES_1_004_NYC_2
id_s,ACCESORIES_1_004_BOS_1,ACCESORIES_1_004_BOS_2,ACCESORIES_1_004_NYC_2
region,Boston,Boston,New York
store_code,BOS_1,BOS_2,NYC_2
store,South_End,Roxbury,Harlem
department,ACCESORIES_1,ACCESORIES_1,ACCESORIES_1
category,ACCESORIES,ACCESORIES,ACCESORIES
item,ACCESORIES_1_004,ACCESORIES_1_004,ACCESORIES_1_004
Monthly_qty,4,10,11


#  Analysis tablas Sales_Calendar(agrupadas) y Precios

In [ ]:
len(gdf_s_C)

1362319

In [ ]:
print('Head')
print(gdf_s_C.head(3))
print('Tail')
print(gdf_s_C.tail(3))

Head
  yearmonth                             id                    id_s    region  \
0    201101  201101_ACCESORIES_1_004_BOS_1  ACCESORIES_1_004_BOS_1    Boston   
1    201101  201101_ACCESORIES_1_004_BOS_2  ACCESORIES_1_004_BOS_2    Boston   
2    201101  201101_ACCESORIES_1_004_NYC_2  ACCESORIES_1_004_NYC_2  New York   

  store_code      store    department    category              item  \
0      BOS_1  South_End  ACCESORIES_1  ACCESORIES  ACCESORIES_1_004   
1      BOS_2    Roxbury  ACCESORIES_1  ACCESORIES  ACCESORIES_1_004   
2      NYC_2     Harlem  ACCESORIES_1  ACCESORIES  ACCESORIES_1_004   

   Monthly_qty  
0            4  
1           10  
2           11  
Tail
        yearmonth                              id                     id_s  \
1362316    201604  201604_SUPERMARKET_3_827_PHI_1  SUPERMARKET_3_827_PHI_1   
1362317    201604  201604_SUPERMARKET_3_827_PHI_2  SUPERMARKET_3_827_PHI_2   
1362318    201604  201604_SUPERMARKET_3_827_PHI_3  SUPERMARKET_3_827_PHI_3   

   

In [ ]:
 print('Head')
 print(gdf_p2.head(3))
 print('Tail')
 print(gdf_p2.tail(3))#Tenemos precios disponibles de 201101 a 201606, en cuanto a las ventas, hay registros de 201105 a 201617. Con lo cual no habrá problemas con juntar las tablas

Head
  yearmonth                             id  average_price
0    201101  201101_ACCESORIES_1_004_BOS_1         5.7722
1    201101  201101_ACCESORIES_1_004_BOS_2         5.7722
2    201101  201101_ACCESORIES_1_004_NYC_2         5.7722
Tail
        yearmonth                              id  average_price
1598936    201606  201606_SUPERMARKET_3_827_PHI_1            1.2
1598937    201606  201606_SUPERMARKET_3_827_PHI_2            1.2
1598938    201606  201606_SUPERMARKET_3_827_PHI_3            1.2


In [ ]:
print('Precios nulos', gdf_p2.isnull().sum())
print('Ventas_Calendario nulos', gdf_s_C.isnull().sum())

Precios nulos yearmonth        0
id               0
average_price    0
dtype: int64
Ventas_Calendario nulos yearmonth      0
id             0
id_s           0
region         0
store_code     0
store          0
department     0
category       0
item           0
Monthly_qty    0
dtype: int64


# Join Sales_Calendar y Precios

In [ ]:
# unimos los datos de precios
df_s_c_p = gdf_s_C.merge(gdf_p2[['id', 'average_price']], left_on='id', right_on='id', how='left')


In [ ]:
df_s_c_p.isnull().sum()

,0
yearmonth,0
id,0
id_s,0
region,0
store_code,0
store,0
department,0
category,0
item,0
Monthly_qty,0


In [ ]:
df_s_c_p=df_s_c_p.drop(columns=['id'])

In [ ]:
df_s_c_p.head(2).T

,0,1
yearmonth,201101,201101
id_s,ACCESORIES_1_004_BOS_1,ACCESORIES_1_004_BOS_2
region,Boston,Boston
store_code,BOS_1,BOS_2
store,South_End,Roxbury
department,ACCESORIES_1,ACCESORIES_1
category,ACCESORIES,ACCESORIES
item,ACCESORIES_1_004,ACCESORIES_1_004
Monthly_qty,4,10
average_price,5.7722,5.7722


#Guardar clean data

In [ ]:
path='/content/drive/MyDrive/TFM/OUTPUTS_FINALES_CSV'

In [ ]:
df_s_c_p.to_csv(path+'/df_s_c_p_Y_M.csv', index=False)

#  PREPROCESSING(YEAR/Week - Groupby)
El DS resultante de este preprocessing se ha utilizado como origen de datos para el calculo del Forecast


# Librerias

In [ ]:
import sys
ENV_COLAB = 'google.colab' in sys.modules
if ENV_COLAB:
      !pip install catboost
      !pip install ipywidgets
      !pip install ydata_profiling
      !jupyter nbextension enable --py widgetsnbextension
      print('Environment: Google Colab')

In [ ]:
import sys
import time
import warnings

# imports best practice pandas
import os
import numpy as np
import pandas as pd

# imports best practice sklearn
import sklearn
from sklearn import set_config

from sklearn.tree import DecisionTreeClassifier

# transformers
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder

# pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# model selection e hiperparameters optimization
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, RepeatedKFold, KFold

from sklearn.metrics import accuracy_score

#Plot and profiling
import matplotlib.pyplot as plt
import seaborn as sns
from ydata_profiling import ProfileReport
from catboost import CatBoostClassifier, Pool
from ydata_profiling import ProfileReport

pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows', None)

import warnings
warnings.filterwarnings("ignore")

# Importar train data

In [ ]:
ENV_COLAB = 'google.colab' in sys.modules
if ENV_COLAB:
  from google.colab import drive
  drive.mount('/content/drive')
else:
  print('Environment: Local')

In [ ]:
ENV_COLAB = 'google.colab' in sys.modules
path='/content/drive/MyDrive/TFM/CSV'
if ENV_COLAB:
  df_p = pd.read_csv(path+'/item_prices.csv')
  df_s=pd.read_csv(path+'/item_sales.csv')
  df_c=pd.read_csv(path+'/daily_calendar_with_events.csv')
else:
  df_p = pd.read_csv(r'CSV/item_prices.csv')
  df_s=pd.read_csv(r'CSV/item_sales.csv')
  df_c=pd.read_csv(r'CSV/daily_calendar_with_events.csv')

# EDA: Analisis Exploratorio

# EDA/Preprocessing: Tabla de Precios

In [ ]:
# Hacemos un Profiling para la tabla de precios

profile_Calendar = ProfileReport(df_p, title='Profiling Report - Price', minimal=True)
profile_Calendar.to_notebook_iframe()

In [ ]:
df_p2=df_p.copy()

In [ ]:
df_p2.isnull().sum()

In [ ]:
df_p2[df_p2['yearweek'].isnull()].head(3)

In [ ]:
#Hay mas de una semana seguida con valores nulos, con lo cual hay que buscar una manera de imputarlos de manera consecutiva
df_p2.iloc[147:158]

In [ ]:
last_valid_value = None
for i in df_p2.index:
  if pd.isnull(df_p2.loc[i, 'yearweek']):
    if last_valid_value:
      df_p2.loc[i, 'yearweek'] = last_valid_value + 1
      last_valid_value += 1 # incremento para imputar los nulos consecutivos y sumar 1 semana al valor anterior
    else:
      # Como el primer registro de la tabla no es nulo, no hace falta hacer ninguna acción.
      pass
  else:
    last_valid_value = df_p2.loc[i, 'yearweek']

In [ ]:
#limpiamos los 0 de la columna de Y/W
df_p2['yearweek'] = df_p2['yearweek'].apply(lambda x: str(x).replace('.0', '') if pd.notnull(x) else x).astype(int)

In [ ]:
# Convertir 'yearweek' en objetos datetime, asumiendo que cada semana comienza el lunes
df_p2['date'] = pd.to_datetime(df_p2['yearweek'].astype(str) + '1', format='%Y%W%w') - pd.Timedelta(days=2) # # Restar 2 días para obtener el sábado anterior, con el analisi de las ventas hemos detectado que las semanas de Yearweek empiezan el sabado

# Extraer mes y año
df_p2['month'] = df_p2['date'].dt.strftime('%m')
df_p2['year'] = df_p2['date'].dt.strftime('%Y')

# Crear columna 'yearmonth'
df_p2['yearmonth'] = df_p2['year'] + df_p2['month']
df_p2.drop(columns=['date','year','month'], inplace=True)

In [ ]:
df_p2 = df_p2.drop_duplicates(keep='first')

In [ ]:
df_p2.isnull().sum()

In [ ]:
df_p2['id']=df_p2['yearweek'].astype(str) +'_'+df_p2['item'].astype(str) +'_'+ df_p2['store_code'].astype(str) #creamos una pk para unir con precios y calendario
df_p2 = df_p2.sort_values('yearweek')

In [ ]:
df_p2.duplicated().sum()

## Group By Precios

In [ ]:
df_p2.head(2).T

In [ ]:
 gdf_p2 = df_p2.groupby(['yearweek', 'id']).agg(
     average_price = ('sell_price', np.mean)
 ).reset_index()


In [ ]:
gdf_p2.head(3).T

In [ ]:
gdf_p2.isnull().sum()

# EDA/Preprocessing: CALENDAR

In [ ]:
# Hacemos un Profiling para la tabla de precios
profile_Calendar = ProfileReport(df_c, title='Profiling Report - Calendar', minimal=True)
profile_Calendar.to_notebook_iframe()

In [ ]:
# borramos la columna EVENTOS 0 relevnate
df_c.drop(columns=['event'], inplace=True)

In [ ]:
# creamos copia para el caledario
df_c1 = df_c.copy()

In [ ]:
# Convertir 'yearweek' en objetos datetime, asumiendo que cada semana comienza el domingo

df_c1['date'] = pd.to_datetime(df_c1['date'])

# Extraer month y year
df_c1['month'] = df_c1['date'].dt.strftime('%m')
df_c1['year'] = df_c1['date'].dt.strftime('%Y')

# Crear columna 'yearmonth'
df_c1['yearmonth'] = df_c1['year'] + df_c1['month']

# Pasar 'yearweek' a datetime
df_c1['date_yw'] = pd.to_datetime(df_c1['date'])

# Calcular el sábado anterior
df_c1['date_yw'] = df_c1['date_yw'] + pd.offsets.Week(weekday=1)

# Extraer año y número de semana (con el sábado como inicio de la semana)
df_c1['yearweek'] = df_c1['date_yw'].dt.strftime('%Y%W').astype(int)


df_c1.drop(columns=['year','month'], inplace=True)

In [ ]:
print(df_c1.head(5))
print(df_c1.tail(5))

In [ ]:
df_c1.isnull().sum()

# EDA SALES

In [ ]:
df_s1 = df_s.copy()

In [ ]:
# hacemos una lista que solo incluya las columnas que empiezan con "d_" y las transponemos para hacer el reshape del dataframe para traer las columnas "d" y su valor

cols = [col for col in df_s1.columns if col.startswith('d_')] # creamos la lista

df_s1 = df_s1.melt(id_vars=[col for col in df_s1.columns if col not in cols],
                   value_vars=cols,
                   var_name='d',
                   value_name='quantity')

In [ ]:
# Hacemos un Profiling para la tabla de precios
profile_Calendar = ProfileReport(df_s1, title='Profiling Report - Sales', minimal=True)
profile_Calendar.to_notebook_iframe()

In [ ]:
# En el profile report vemos que, al hacer el "unpivot" de las columnas de "d_", tenemos registros con venta =0. Quitamos los registros con venta=0
df_s1 = df_s1[df_s1['quantity']!=0]

In [ ]:
#El df por dias pesa demasiado para seguir trabajando por dias, por ello, despues de unirlo con el calendario, habrá que agrupar por semanas
df_s1.info()

# JOINS

# Join Sales + Calendar

In [ ]:
# hacemos merge para las ventas y el calendario
df_s_C = df_s1.merge(df_c1[['d', 'date_yw','yearweek','yearmonth']], left_on='d', right_on='d', how='left')

In [ ]:
df_s_C.head(3).T

In [ ]:
df_s_C.rename(columns={'id': 'id_s', 'date_yw': 'date'}, inplace=True)

In [ ]:
print('Sales_Calendar', df_s_C.shape)


In [ ]:
#Creamos una pk para unir con precios
df_s_C['id']= df_s_C['yearweek'].astype(str) + '_' + df_s_C['id_s'].astype(str)

In [ ]:
df_s_C.head(3).T

## Groupby Sales+Calendar(by year_week)

In [ ]:
gdf_s_C = df_s_C.groupby(['yearweek','yearmonth', 'id','id_s','region','store_code','store','department','category','item']).agg(
     weekly_qty= ('quantity', np.sum)
 ).reset_index()



In [ ]:
gdf_s_C.head(3).T

# Analysis tablas Sales_Calendar(agrupadas) y Precios

In [ ]:
print('Head')
print(gdf_s_C.head(3))
print('Tail')
print(gdf_s_C.tail(3))

In [ ]:
print('Head')
print(gdf_p2.head(3))
print('Tail')
print(gdf_p2.tail(3))  #Tenemos precios disponibles de 201105 a 201625, en cuanto a las ventas, hay registros de 201105 a 201617. Con lo cual no habrá problemas con juntar las tablas

In [ ]:
print('Precios nulos', gdf_p2.isnull().sum())
print('Ventas_Calendario nulos', gdf_s_C.isnull().sum())

# Join Sales_Calendar y Precios

In [ ]:
# unimos los datos de precios
df_s_c_p = gdf_s_C.merge(gdf_p2[['id', 'average_price']], left_on='id', right_on='id', how='left')
df_s_c_p

In [ ]:
df_s_c_p.head(2).T

In [ ]:
df_s_c_p.isnull().sum()

In [ ]:
df_s_c_p=df_s_c_p.drop(columns=['id'])

#Guardar clean data

In [ ]:
path='/content/drive/MyDrive/TFM/OUTPUTS_FINALES_CSV'

In [ ]:
df_s_c_p.to_csv(path+'/df_s_c_p_Y_W.csv', index=False)